In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Data preprocessing

In [2]:
df = pd.read_csv('path_to_your_file/data_g.csv')

In [3]:
df = df.reset_index(drop=True)

num_timesteps = 60
num_patients = len(df) // num_timesteps
patients = np.repeat(np.arange(num_patients), num_timesteps)
df['patient_id'] = patients

columns = list(df.columns)

In [19]:
df

,recovery_flags,cancer_volume,chemo_application,radio_application,sequence_lengths,chemo_dosage,radio_dosage,patient_types,index,patient_id
0,0.0,4.893958,0.0,0.0,59.0,0.000000,0.0,2,0,0
1,0.0,4.966752,0.0,0.0,59.0,0.000000,0.0,2,0,0
2,0.0,5.127451,0.0,0.0,59.0,0.000000,0.0,2,0,0
3,0.0,5.198159,0.0,0.0,59.0,0.000000,0.0,2,0,0
4,0.0,5.392657,0.0,0.0,59.0,0.000000,0.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...
599995,0.0,0.293599,0.0,0.0,59.0,0.000078,0.0,2,9999,9999
599996,0.0,0.313837,0.0,0.0,59.0,0.000039,0.0,2,9999,9999
599997,0.0,0.335010,0.0,0.0,59.0,0.000019,0.0,2,9999,9999
599998,0.0,0.362887,0.0,0.0,59.0,0.000010,0.0,2,9999,9999


In [4]:
len(set(patients))

10000

In [6]:
modifed_df = pd.DataFrame(columns=columns)
unique_patients = set(patients)
for patient in unique_patients:
    mask = df['patient_id'] == patient
    filtered_df = df.loc[mask, :].copy().reset_index()
    
    sequence_lengths = int(filtered_df["sequence_lengths"][0])
    filtered_df = filtered_df[:sequence_lengths + 1]
        
    modifed_df = pd.concat([modifed_df, filtered_df], ignore_index=True)
    
    
modifed_df = modifed_df.reset_index(drop=True)


In [1]:
modifed_df

# Here we have preprocessed data and stored it in modifed_df. We then save it to a csv file and read it back in. 
# This is to ensure that we have the same data as the original notebook. 
# Further, we work with the name df for the rest of the notebook.

modifed_df.to_csv('modified_data.csv', index=False)
df = pd.read_csv('path_to_your_file/modified_data.csv')

NameError: name 'modifed_df' is not defined

In [17]:
train, test = train_test_split(modifed_df, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=0.25, random_state=42)

treatment_policies = {
    'no_treatment': {'chemo_application': 0, 'radio_application': 0},
    'chemotherapy_only': {'chemo_application': 1, 'radio_application': 0},
    'radiotherapy_only': {'chemo_application': 0, 'radio_application': 1},
    'chemo_radiotherapy': {'chemo_application': 1, 'radio_application': 1}
}

# The dosages of chemotherapy and radiotherapy are chosen randomly between its minimal and maximal values.
dosages = {
    'chemotherapy_only': {'chemo_dosage': 9, 'radio_dosage': 0}, 
    'radiotherapy_only': {'radio_dosage': 2, 'chemo_dosage': 0},
    'chemo_radiotherapy': {'chemo_dosage': 9, 'radio_dosage': 2},
    'no_treatment': {'chemo_dosage': 0, 'radio_dosage': 0}
}

In [18]:
def preprocess_data(df):
    df['t'] = df.groupby('patient_id').cumcount() + 1
    df = df.rename(columns={'recovery_flags': 'event'})
    df = pd.get_dummies(df, columns=['patient_types'], drop_first=True)
    return df

train_processed = preprocess_data(train)
valid_processed = preprocess_data(valid)
test_processed = preprocess_data(test)

train_processed.head()

/var/folders/cb/z2s3pcxj12zbvxns0bbk2tbc0000gn/T/ipykernel_58762/3772311057.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=['patient_types'], drop_first=True)
/var/folders/cb/z2s3pcxj12zbvxns0bbk2tbc0000gn/T/ipykernel_58762/3772311057.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=['patient_types'], drop_first=True)
/var/folders/cb/z2s3pcxj12zbvxns0bbk2tbc0000gn/T/ipykernel_58762/3772311057.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=['patient_types'], drop_first=True)


,event,cancer_volume,chemo_application,radio_application,sequence_lengths,chemo_dosage,radio_dosage,index,patient_id,level_0,t,patient_types_2,patient_types_3
74569,0.0,0.421393,0.0,0.0,59.0,0.000174,0.0,1272,1272,76376.0,1,1,0
107981,0.0,6.970065,0.0,0.0,59.0,0.481024,0.0,1846,1846,110779.0,1,1,0
538765,0.0,5.339738,0.0,0.0,59.0,0.391083,0.0,9222,9222,553336.0,1,1,0
292154,0.0,0.387902,0.0,0.0,59.0,0.000000,0.0,5000,5000,300009.0,1,0,0
189213,0.0,1.980024,0.0,0.0,59.0,2.504965,0.0,3231,3231,193885.0,1,0,0


In [19]:
def estimate_probability(df, X_columns, y_column):
    X = df[X_columns]
    y = df[y_column]

    return model.predict_proba(X)[:, 1]

def calculate_weights(train_df, treatment_policy, dosages):
    train_df = train_df.copy()
    
    # Get the dosage for the current treatment policy
    chemo_dosage = dosages.get(treatment_policy, {}).get('chemo_dosage', 0)
    radio_dosage = dosages.get(treatment_policy, {}).get('radio_dosage', 0)

    small_constant = 1e-10  # small constant to avoid division by zero
    chemo_prob = np.ones(train_df.shape[0]) * (chemo_dosage / 100 + small_constant) # scaling down as original probability was between 0 and 1
    radio_prob = np.ones(train_df.shape[0]) * (radio_dosage / 100 + small_constant) # scaling down as original probability was between 0 and 1

    censoring_prob = np.ones_like(chemo_prob)
    
    # Calculate IPTW and censoring weights
    train_df["iptw"] = np.where(train_df["chemo_application"] == 1, 1 / chemo_prob, 1 / (1 - chemo_prob))
    train_df["iptw"] *= np.where(train_df["radio_application"] == 1, 1 / radio_prob, 1 / (1 - radio_prob))
    train_df["censoring_weight"] = 1 / censoring_prob
    
    return train_df

def fit_lstm(train_df, valid_df, treatment_policy, dosages):
    train_df_weighted = calculate_weights(train_df, treatment_policy, dosages)
    
    model_list = []
    max_t = train_df_weighted["t"].max()
    num_features = 5
    
    for t in range(1, max_t + 1):
        train_t = train_df_weighted[train_df_weighted["t"] == t]
        X_train = train_t[["cancer_volume", "patient_types_3", "patient_types_2", "chemo_application", "radio_application"]].values
        y_train = train_t["event"].values
        sample_weight = train_t["iptw"] * train_t["censoring_weight"]
        
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        
        if len(np.unique(y_train)) > 1:
            model = Sequential()
            model.add(LSTM(32, input_shape=(1, num_features)))
            model.add(Dense(1, activation='sigmoid'))
            model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')

            model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, sample_weight=sample_weight)
        else:
            model = None
        model_list.append(model)

    y_pred_proba_list = []
    for t, model in enumerate(model_list):
        valid_t = valid_df[valid_df["t"] == t + 1]
        X_valid = valid_t[["cancer_volume", "patient_types_3", "patient_types_2", "chemo_application", "radio_application"]].values
        X_valid = X_valid.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
        if model is not None:
            y_pred_proba = model.predict(X_valid).flatten()
        else:
            y_pred_proba = np.full(len(X_valid), train_df_weighted.loc[train_df_weighted["t"] == t + 1, "event"].iloc[0])
        y_pred_proba_list.append(y_pred_proba)

    y_pred_proba = np.concatenate(y_pred_proba_list)
    
    return y_pred_proba

# Calculate predicted probabilities for each treatment policy
policy_results = {}
for policy_name in treatment_policies.keys():
    y_pred_proba = fit_lstm(train_processed, valid_processed, policy_name, dosages)
    policy_results[policy_name] = y_pred_proba


# Compare the predicted probabilities for each treatment policy
print("Treatment Policy Results:")
for policy_name, result in policy_results.items():
    print(f"{policy_name}: {result}")

2023-05-11 22:53:25.953369: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 235ms/step
Treatment Policy Results:
no_treatment: [nan nan nan ...  0.  0.  0.]
chemotherapy_only: [0.00107896 0.00083469 0.00050387 ... 0.         0.         0.        ]
radiotherapy_only: [0.00198798 0.00167115 0.00075695 ... 0.         0.         0.        ]
chemo_radiotherapy: [0.00126955 0.00148473 0.00032625 ... 0.         0.         0.        ]


In [21]:
# Calculate the average predicted probability of recovery for each treatment policy
avg_probabilities = {}
for policy_name, probabilities in policy_results.items():
    avg_prob = np.mean(probabilities)
    avg_probabilities[policy_name] = avg_prob

# Print the average predicted probabilities
print("Average Predicted Probabilities of Recovery:")
for policy_name, avg_prob in avg_probabilities.items():
    print(f"{policy_name}: {avg_prob:.6f}")

Average Predicted Probabilities of Recovery:
no_treatment: nan
chemotherapy_only: 0.004034
radiotherapy_only: 0.007314
chemo_radiotherapy: 0.001590
